## YOLO - Intruder Detection

### Required Dependencies

In [1]:
# Supress warnings for entire notebook
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import the video processing and deep learning modules
import cv2
import numpy as np
import os
import threading
from ultralytics import YOLO
from datetime import datetime
import time
import json

In [3]:
# # Get firebase backend storage functions
# import firebase_admin
# from firebase_admin import credentials, storage, firestore

### YOLO - Setup

In [4]:
# Load YOLO model
yolo_model = YOLO("yolov8n.pt")  # Ensure you have the model downloaded

In [5]:
# Create a folder for saving intruder images
INTRUDER_FOLDER = "intruders"
os.makedirs(INTRUDER_FOLDER, exist_ok=True)

In [6]:
# Intruder detection state
state = 'stop'   # 'stop', 'start'
last_intruder_time = 0    # intruder time tracker
follow_update = False     # update flag to tick intruder time

# Counter for burst and follow-up frames
intruder_frames = 0

# Flag to prevent multiple burst threads running at the same time
burst_in_progress = False

### JSON - Setup

In [7]:
# BASE_URI where images are served by Flask server
# JSON_FILE to store metadata
JSON_FILE = "intruderImages.json"
BASE_URI = "http://172.20.10.3:8000/images/"  # Update if needed

### JSON - Functions

In [8]:
def get_existing_data():
    """ Load existing data from JSON if file exists. """
    if os.path.exists(JSON_FILE):
        with open(JSON_FILE, "r") as file:
            return json.load(file)
    return []

In [9]:
def write_data_to_json(data):
    """ Write updated data to JSON file. """
    with open(JSON_FILE, "w") as file:
        json.dump(data, file, indent=4)

In [10]:
def generate_entry(file_name, entry_id):
    """ Generate entry for new image. """
    uri = BASE_URI + file_name
    timestamp = datetime.now().strftime("%Y-%m-%d %I:%M:%S %p")

    return {
        "id": str(entry_id),
        "uri": uri,
        "name": f"Intruder {entry_id}",
        "timestamp": timestamp
    }

### YOLO - Functions

In [11]:
# Function to detect people using YOLO
def detect_persons(frame):
    results = yolo_model(frame)  # Run YOLO
    persons = []

    for result in results:
        for box, class_id, conf in zip(result.boxes.xyxy, result.boxes.cls, result.boxes.conf):  # Extract confidence score
            if int(class_id) == 0:  # Class 0 in COCO is "person"
                x1, y1, x2, y2 = map(int, box[:4])
                persons.append({
                    "bbox": (x1, y1, x2, y2),  # Bounding box
                    "confidence": conf.item()   # Confidence score
                })

    return persons

In [12]:
# Helper function to save a frame
def save_intruder_frame(frame, prefix=""):
    # Save files to local disk
    filename = f"{INTRUDER_FOLDER}/Intruder_{prefix}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.jpg"
    cv2.imwrite(filename, frame)
    print(f"Saved: {filename}")
    
    # Generate and Update the metadata
    existing_data = get_existing_data()
    new_id = len(existing_data) + 1
    entry = generate_entry(filename, new_id)
    existing_data.append(entry)
    write_data_to_json(existing_data)
    print(f"New file detected: {filename}, entry added to JSON.")
    
    # Optional: Call FIREBASE cloud upload function here
    # handle_intruder_detection(filename, confidence)

In [13]:
# First Burst Function - 5 photos immediately, 300ms apart
def first_burst(frame):
    global state, intruder_frames, burst_in_progress
    
    if state != 'stop' or burst_in_progress:
        return  # Already capturing or burst in progress
    
    state = 'start'
    intruder_frames = 0
    burst_in_progress = True  # Set the flag to indicate burst is in progress
    print("Intruder detected - Starting first burst capture")

    # Background thread for burst capture
    def capture_burst():
        global intruder_frames, burst_in_progress
        
        for i in range(5):
            save_intruder_frame(frame, f"{i+1}_burst")
            intruder_frames += 1
            time.sleep(0.3)  # This sleep only affects the burst thread, not the main loop

        burst_in_progress = False  # Reset flag after burst capture is complete

    # Start a thread for the burst capture
    threading.Thread(target=capture_burst, daemon=True).start()

In [14]:
# Follow-up Function - 1 photo every 2 seconds
def follow_up(frame):
    global state, last_intruder_time, intruder_frames, follow_update
    
    if state != 'start':
        return  # No active capture
    
    if time.time() - last_intruder_time >= 2:  # 2 seconds interval
        save_intruder_frame(frame, "followup")
        intruder_frames += 1
        last_intruder_time = time.time()

In [15]:
# Exit Function - stop after 5 seconds of no intruder detected
def exit_capture():
    global state, intruder_frames, follow_update
    
    if state != 'start':
        return
    
    print("No intruder for 5 seconds - Stopping capture")
    state = 'stop'
    intruder_frames = 0
    follow_update = False

### Firebase - Functions

In [16]:
# # Upload image to Firebase Storage
# def upload_image(image_path):
#     bucket = storage.bucket()
#     blob = bucket.blob(f"intruders/{os.path.basename(image_path)}")
#     blob.upload_from_filename(image_path)
#     blob.make_public()  # Makes the image URL publicly accessible (optional)

#     return blob.public_url

In [17]:
# # Save Metadata to Firestore
# def save_metadata(image_url, confidence):
#     doc_ref = db.collection('intruder_logs').add({
#         'imageUrl': image_url,
#         'timestamp': datetime.now().isoformat(),
#         'confidence': confidence
#     })

In [18]:
# # Full Function to Call from Detection Code
# def handle_intruder_detection(image_path, confidence):
#     image_url = upload_image(image_path)
#     save_metadata(image_url, confidence)
#     print(f"Intruder saved: {image_url}")

### Firebase - Setup

In [19]:
# # Initialize Firebase App
# cred = credentials.Certificate("firebase_key.json")
# firebase_admin.initialize_app(cred, {
#     'storageBucket': 'intruder-detection-syste-3e8f1.appspot.com'
# })
# db = firestore.client()

### Intruder Detection Loop - Frame by Frame Analysis

In [20]:
# Initialize webcam
cap = cv2.VideoCapture(0)

In [21]:
# Start analyzing the frames until quit
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    persons = detect_persons(frame)
    
    if persons:  # Intruder Detected
#         print("PERSON DETECTED ...")
        if follow_update == False:
            last_intruder_time = time.time()
            follow_update = True

        if state == 'stop':
#             print("FIRST BURST ...")
            first_burst(frame)  # Trigger burst when intruder detected for first time
        
        # Draw boxes and labels
        for person in persons:
            x1, y1, x2, y2 = person["bbox"]
            confidence = person["confidence"]

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            text = f"Intruder Detected: {confidence:.2f}"
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        # Continuous follow-up capture if burst is done
        if state == 'start' and intruder_frames >= 5 and burst_in_progress == False:
            follow_up(frame)

    else:  # No intruder detected
        print("...NO PERSON DETECTED!...")
        if state == 'start' and time.time() - last_intruder_time > 5:
            exit_capture()

    # Display the live feed with annotations
    cv2.imshow("Intruder Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    
cap.release()
cv2.destroyAllWindows()


0: 480x640 3 persons, 209.3ms
Speed: 24.3ms preprocess, 209.3ms inference, 4.5ms postprocess per image at shape (1, 3, 480, 640)
Intruder detected - Starting first burst capture
Saved: intruders/Intruder_1_burst_2025-03-11_11-01-49.jpg
New file detected: intruders/Intruder_1_burst_2025-03-11_11-01-49.jpg, entry added to JSON.

0: 480x640 3 persons, 109.8ms
Speed: 4.4ms preprocess, 109.8ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 108.1ms
Speed: 5.1ms preprocess, 108.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Saved: intruders/Intruder_2_burst_2025-03-11_11-01-49.jpg
New file detected: intruders/Intruder_2_burst_2025-03-11_11-01-49.jpg, entry added to JSON.

0: 480x640 4 persons, 1 laptop, 98.7ms
Speed: 2.5ms preprocess, 98.7ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 131.6ms
Speed: 18.0ms preprocess, 131.6ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 6


0: 480x640 4 persons, 95.3ms
Speed: 2.1ms preprocess, 95.3ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 96.6ms
Speed: 2.7ms preprocess, 96.6ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 1 bottle, 101.8ms
Speed: 2.4ms preprocess, 101.8ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 100.9ms
Speed: 2.5ms preprocess, 100.9ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 92.4ms
Speed: 2.3ms preprocess, 92.4ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 1 bottle, 98.0ms
Speed: 2.8ms preprocess, 98.0ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 97.3ms
Speed: 2.2ms preprocess, 97.3ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 83.7ms
Speed: 2.4ms preprocess, 83.7ms inference, 1.7ms postproc